#Recurrent Neural Network For Weather Prediction

###Start spark application and set configuration

In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1575218563911_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f
{ "conf":{
"spark.pyspark.python": "python3",
"spark.pyspark.virtualenv.enabled": "true",
"spark.pyspark.virtualenv.type":"native",
"spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1575218563911_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1575218563911_0006,pyspark,idle,Link,Link,✔


###install python libraries

In [3]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("sklearn")
sc.install_pypi_package("tensorflow")
sc.install_pypi_package("elephas")
sc.install_pypi_package("keras")
sc.install_pypi_package("request")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/52/3f/f6a428599e0d4497e1595030965b5ba455fd8ade6e977e3c819973c4b41d/pandas-0.25.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

Processing /var/lib/livy/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074/sklearn-0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a0/c5/d2238762d780dde84a20b8c761f563fe882b88c5a5fb03c056547c442a19/scikit_learn-0.21.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8f/42/155696f85f344c066e17af287359c9786b436b1bf86029bb3411283274f3/joblib-0.14.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/54/18/d7c101d5e93b6c78dc206fcdf7bd04c1f8138a7b1a93578158fa3b132b08/scipy-1.3.3-cp36-cp36m-manylinux1_x86_64.whl

  Using cached https://files.

###Retrieving data from Dark Sky API

In [4]:
import csv
import requests
import pandas as pd
import time
def retrieve_data():
    def converTime(dt):
        timeArray = time.strptime(dt, "%Y-%m-%d")
        timestamp = time.mktime(timeArray).__int__()
        return timestamp


    def converBack(ts):
        time_format = time.localtime(ts)
        dt = time.strftime("%Y-%m-%d %H:%M", time_format)
        return str(dt)


    def plusOneDay(dt):
        end_date = pd.to_datetime(dt) + pd.DateOffset(days=1)
        return str(end_date)[0:10]

    # Using Loop to control the message number of Temperature
    start_date = "2018-11-1"
    day_num = 150
    output_file = open(
        file="/result/dataset",
        mode="wt",
        newline="")
    csv_writer = csv.writer(output_file)
    csv_writer.writerow(["time", "summary","icon","precipIntensity","precipProbability","precipType","temperature",
                         "apparentTemperature","dewPoint","humidity","pressure","windSpeed","windGust","windBearing",
                         "cloudCover","uvIndex","visibility","ozone"])
    for i in range(day_num):
        time_stamp = converTime(start_date)
        # Call Dark Sky api to retrieve the temperature of specific date
        DALLAS = 32.7767, -96.7970
        api_key = "4650359a111dac80aafb7fa8d846f0cb"
        url_template = "https://api.darksky.net/forecast/{}/{},{},{}?exclude=currently,flags,daily"
        request_url = url_template.format(api_key, DALLAS[0], DALLAS[1], time_stamp)
        response = requests.get(request_url)
        json_data = response.json()
        for i in range(len(json_data["hourly"]["data"])):
            # Save the result as CSV rows
            time_column = json_data["hourly"]["data"][i]["time"]
            converted_time = converBack(time_column)
            summary_column = "None"
            icon_column = "None"
            precipIntensity_column = "None"
            precipProbability_column = "None"
            precipType_column = "None"
            temperature_column = "None"
            apparentTemperature_column = "None"
            dewPoint_column = "None"
            humidity_column = "None"
            pressure_column = "None"
            windSpeed_column = "None"
            windGust_column = "None"
            windBearing_column = "None"
            cloudCover_column = "None"
            uvIndex_column = "None"
            visibility_column = "None"
            if("summary" in json_data["hourly"]["data"][i]):
                summary_column = json_data["hourly"]["data"][i]["summary"]
            if ("icon" in json_data["hourly"]["data"][i]):
                icon_column = json_data["hourly"]["data"][i]["icon"]
            if ("precipIntensity" in json_data["hourly"]["data"][i]):
                precipIntensity_column = json_data["hourly"]["data"][i]["precipIntensity"]
            if ("precipProbability" in json_data["hourly"]["data"][i]):
                precipProbability_column = json_data["hourly"]["data"][i]["precipProbability"]
            if ("precipType" in json_data["hourly"]["data"][i]):
                precipType_column = json_data["hourly"]["data"][i]["precipType"]
            if ("temperature" in json_data["hourly"]["data"][i]):
                temperature_column = json_data["hourly"]["data"][i]["temperature"]
            if ("apparentTemperature" in json_data["hourly"]["data"][i]):
                apparentTemperature_column = json_data["hourly"]["data"][i]["apparentTemperature"]
            if ("dewPoint" in json_data["hourly"]["data"][i]):
                dewPoint_column = json_data["hourly"]["data"][i]["dewPoint"]
            if ("humidity" in json_data["hourly"]["data"][i]):
                humidity_column = json_data["hourly"]["data"][i]["humidity"]
            if ("pressure" in json_data["hourly"]["data"][i]):
                pressure_column = json_data["hourly"]["data"][i]["pressure"]
            if ("windSpeed" in json_data["hourly"]["data"][i]):
                windSpeed_column = json_data["hourly"]["data"][i]["windSpeed"]
            if ("windGust" in json_data["hourly"]["data"][i]):
                windGust_column = json_data["hourly"]["data"][i]["windGust"]
            if ("windBearing" in json_data["hourly"]["data"][i]):
                windBearing_column = json_data["hourly"]["data"][i]["windBearing"]
            if ("cloudCover" in json_data["hourly"]["data"][i]):
                cloudCover_column = json_data["hourly"]["data"][i]["cloudCover"]
            if ("uvIndex" in json_data["hourly"]["data"][i]):
                uvIndex_column =  json_data["hourly"]["data"][i]["uvIndex"]
            if ("visibility" in json_data["hourly"]["data"][i]):
                visibility_column = json_data["hourly"]["data"][i]["visibility"]
            csv_writer.writerow([converted_time, summary_column,icon_column,precipIntensity_column,precipProbability_column,precipType_column,temperature_column,
                                 apparentTemperature_column,dewPoint_column,humidity_column,pressure_column,windSpeed_column,windGust_column,windBearing_column,
                                 cloudCover_column,uvIndex_column,visibility_column])
        # End modification and exit IO execution
        start_date = plusOneDay(start_date)
    output_file.close()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

###Using spark dataframe to prepocess data

In [5]:
weather_rawdata = spark.read.option("header","true"). option("inferSchema","true").csv("s3://weatherprediction/dataset")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
weather_rawdata.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------+-------------------+---------------+-----------------+----------+-----------+-------------------+--------+--------+--------+---------+--------+-----------+----------+-------+----------+
|            time|      summary|               icon|precipIntensity|precipProbability|precipType|temperature|apparentTemperature|dewPoint|humidity|pressure|windSpeed|windGust|windBearing|cloudCover|uvIndex|visibility|
+----------------+-------------+-------------------+---------------+-----------------+----------+-----------+-------------------+--------+--------+--------+---------+--------+-----------+----------+-------+----------+
|2018-11-01 00:00|     Overcast|             cloudy|              0|                0|      None|      53.17|              53.17|   50.37|     0.9|  1010.1|     8.31|    8.31|        315|         1|      0|     9.997|
|2018-11-01 01:00|     Overcast|             cloudy|              0|                0|      None|      53.19|              53.19

In [7]:
raw_data = weather_rawdata.select("time","temperature")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
raw_data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------+
|            time|temperature|
+----------------+-----------+
|2018-11-01 00:00|      53.17|
|2018-11-01 01:00|      53.19|
|2018-11-01 02:00|      52.92|
|2018-11-01 03:00|       52.8|
|2018-11-01 04:00|      52.54|
|2018-11-01 05:00|       52.7|
|2018-11-01 06:00|      52.57|
|2018-11-01 07:00|      51.36|
|2018-11-01 08:00|      50.98|
|2018-11-01 09:00|      51.78|
|2018-11-01 10:00|      54.32|
|2018-11-01 11:00|      57.03|
|2018-11-01 12:00|      58.21|
|2018-11-01 13:00|      59.75|
|2018-11-01 14:00|      60.74|
|2018-11-01 15:00|      60.95|
|2018-11-01 16:00|      61.85|
|2018-11-01 17:00|      61.77|
|2018-11-01 18:00|      60.54|
|2018-11-01 19:00|      58.69|
+----------------+-----------+
only showing top 20 rows

###Implementation of LSTM

In [9]:
from keras import *
from keras import backend as K
from keras.engine.base_layer import Layer
from keras.layers import Dense, RNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.legacy import interfaces

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Using TensorFlow backend.

In [10]:
class CustomLSTMCell(Layer):
    def __init__(self, units_num, **kwargs):
        super(CustomLSTMCell, self).__init__(**kwargs)
        self.units_num = units_num
        self.state_size = (self.units_num, self.units_num)
        self.output_size = self.units_num

    def build(self, input_shape):
        input_dim = input_shape[-1]
        w_shape = (input_dim, self.units_num * 4)
        u_shape = (self.units_num, self.units_num * 4)

        # initialize kernel, recurrent kernal and bias matrix
        self.w = self.add_weight(shape=w_shape, name='matrix_w', initializer='glorot_uniform')

        self.u = self.add_weight(shape=u_shape, name='matrix_u', initializer='orthogonal')

        self.bias = np.zeros(self.units_num * 4)
        self.bias[self.units_num:self.units_num * 2] = 1.

        self.built = True

    def call(self, cell_inputs, cell_states):
        h_t_1 = cell_states[0]
        c_t_1 = cell_states[1]

        temp = K.dot(cell_inputs, self.w) + K.dot(h_t_1, self.u) + self.bias
        temp_i = temp[:, 0:self.units_num]
        temp_f = temp[:, self.units_num:self.units_num * 2]
        temp_c = temp[:, self.units_num * 2:self.units_num * 3]
        temp_o = temp[:, self.units_num * 3:self.units_num * 4]

        i_t = K.sigmoid(temp_i)
        f_t = K.sigmoid(temp_f)
        c_t = f_t * c_t_1 + i_t * K.tanh(temp_c)
        o_t = K.sigmoid(temp_o)

        h_t = o_t * K.tanh(c_t)

        return h_t, [h_t, c_t]

    def get_config(self):
        config = {'units_num': self.units_num}
        base_config = super(CustomLSTMCell, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
class LSTMLayer(RNN):
    @interfaces.legacy_recurrent_support
    def __init__(self, units,
                 activity_regularizer=None,
                 return_sequences=False,
                 return_state=False,
                 go_backwards=False,
                 stateful=False,
                 unroll=False,
                 **kwargs):

        cell = CustomLSTMCell(units)
        super(LSTMLayer, self).__init__(cell,
                                   return_sequences=return_sequences,
                                   return_state=return_state,
                                   go_backwards=go_backwards,
                                   stateful=stateful,
                                   unroll=unroll,
                                   **kwargs)
        self.activity_regularizer = regularizers.get(activity_regularizer)

    def call(self, inputs, mask=None, training=None, initial_state=None):
        self.cell._dropout_mask = None
        self.cell._recurrent_dropout_mask = None
        return super(LSTMLayer, self).call(inputs,
                                      mask=mask,
                                      training=training,
                                      initial_state=initial_state)

    @property
    def units(self):
        return self.cell.units_num

    def get_config(self):
        config = {'units': self.units}
        base_config = super(LSTMLayer, self).get_config()
        del base_config['cell']
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        return cls(**config)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
data_set = raw_data.select("temperature").collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# scale the dataset to [0, 1]
data_scale_tool = MinMaxScaler(feature_range=(0, 1))
data_set = data_scale_tool.fit_transform(data_set)

# split dataset into training set and testing set
training_set_size = int(len(data_set) * 0.67)
testing_set_size = len(data_set) - training_set_size
training_set, testing_set = data_set[0:training_set_size, :], data_set[training_set_size:len(data_set), :]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
import numpy as np
def create_my_data_set(data_set, prev_data_length=1):
    x, y = [], []
    for i in range(len(data_set) - prev_data_length - 1):
        a = data_set[i:(i + prev_data_length), 0]
        x.append(a)
        y.append(data_set[i + prev_data_length, 0])
    return np.array(x), np.array(y)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# preprocess training set and testing set, add previous weather data for LSTM
prev_data_length = 4
num_of_units = 5
training_set_x, training_set_y = create_my_data_set(training_set, prev_data_length)
testing_set_x, testing_set_y = create_my_data_set(testing_set, prev_data_length)

# transfer input into [samples, time steps, features]
training_set_x = np.reshape(training_set_x, (training_set_x.shape[0], 1, training_set_x.shape[1]))
testing_set_x = np.reshape(testing_set_x, (testing_set_x.shape[0], 1, testing_set_x.shape[1]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

###Create and fit the Keras LSTM model

In [20]:
keras_model = Sequential()
keras_model.add(LSTMLayer(num_of_units, input_shape=(1, prev_data_length)))
keras_model.add(Dense(1))
keras_model.compile(loss='mean_squared_error', optimizer='adam')
keras_model.fit(training_set_x, training_set_y, epochs=100, batch_size=1, verbose=2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Epoch 1/100
 - 4s - loss: 0.0084
Epoch 2/100
 - 3s - loss: 0.0020
Epoch 3/100
 - 3s - loss: 9.7263e-04
Epoch 4/100
 - 3s - loss: 8.9389e-04
Epoch 5/100
 - 3s - loss: 8.4495e-04
Epoch 6/100
 - 3s - loss: 7.9297e-04
Epoch 7/100
 - 3s - loss: 7.5029e-04
Epoch 8/100
 - 3s - loss: 6.9619e-04
Epoch 9/100
 - 3s - loss: 6.5966e-04
Epoch 10/100
 - 3s - loss: 6.3533e-04
Epoch 11/100
 - 3s - loss: 6.0745e-04
Epoch 12/100
 - 3s - loss: 5.8878e-04
Epoch 13/100
 - 3s - loss: 5.5760e-04
Epoch 14/100
 - 3s - loss: 5.4482e-04
Epoch 15/100
 - 3s - loss: 5.3117e-04
Epoch 16/100
 - 3s - loss: 5.0825e-04
Epoch 17/100
 - 3s - loss: 5.0511e-04
Epoch 18/100
 - 3s - loss: 5.0642e-04
Epoch 19/100
 - 3s - loss: 5.0105e-04
Epoch 20/100
 - 3s - loss: 5.0317e-04
Epoch 21/100
 - 3s - loss: 4.9458e-04
Epoch 22/100
 - 3s - loss: 4.8804e-04
Epoch 23/100
 - 3s - loss: 4.8000e-04
Epoch 24/100
 - 3s - loss: 4.8228e-04
Epoch 25/100
 - 4s - loss: 4.9003e-04
Epoch 26/100
 - 3s - loss: 4.7975e-04
Epoch 27/100
 - 3s - loss: 4.

In [21]:
# make predictions on training set and testing set
training_set_predict = keras_model.predict(training_set_x)
testing_set_predict = keras_model.predict(testing_set_x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

###Error Result with using Keras model

In [22]:
# calculate root mean squared error
import math
training_set_predict_score = math.sqrt(mean_squared_error(training_set_y, training_set_predict))
print('Train Score: %.5f RMSE' % (training_set_predict_score))
testing_set_predict_score = math.sqrt(mean_squared_error(testing_set_y, testing_set_predict))
print('Test Score: %.5f RMSE' % (testing_set_predict_score))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Train Score: 0.02025 RMSE
Test Score: 0.02110 RMSE

###Using elephas wrap keras and fit the traing dataset in distributed way

In [23]:
from elephas.spark_model import SparkModel
from elephas import optimizers as elephas_optimizers
from elephas.utils.rdd_utils import to_simple_rdd
adagrad = elephas_optimizers.Adagrad()
spark_model = SparkModel(keras_model, frequency='epoch', mode='synchronous', custom_objects={'LSTMLayer':LSTMLayer, 'CustomLSTMCell':CustomLSTMCell})
rdd = to_simple_rdd(sc, training_set_x, training_set_y)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

WARNING

In [24]:
spark_model.fit(rdd, epochs=100, batch_size=1, verbose=2, validation_split=0.1)
training_set_predict_spark=spark_model.predict(training_set_x)
testing_set_predict_spark=spark_model.predict(testing_set_x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

>>> Fit model
>>> Synchronous training complete.

###Error Result with using Elephas

In [25]:
training_set_predict_score = math.sqrt(mean_squared_error(training_set_y, training_set_predict_spark))
print('Train Score: %.5f RMSE' % (training_set_predict_score))
testing_set_predict_score = math.sqrt(mean_squared_error(testing_set_y, testing_set_predict_spark))
print('Test Score: %.5f RMSE' % (testing_set_predict_score))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Train Score: 0.03397 RMSE
Test Score: 0.03414 RMSE